<a href="https://colab.research.google.com/github/cr21/Unsupervised-Machine-Learning-Clustering/blob/main/Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive/MyDrive/6220Data/

/content/drive/MyDrive/6220Data


In [3]:
ls

Exam.ipynb                          soft_clustering_measure.py
mnist_noisy_SAMPLE5000_K20_F31.txt  whole_dataset.txt
sample_dataset.txt


In [4]:
import os
import re
import scipy
from time import time
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import  LatentDirichletAllocation
from sklearn.decomposition import  NMF
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import numpy as np
import nltk
import math
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
mnist_dataset = np.loadtxt('mnist_noisy_SAMPLE5000_K20_F31.txt')

In [86]:
from sklearn.cluster import KMeans, DBSCAN, hierarchical, AgglomerativeClustering, FeatureAgglomeration, SpectralClustering

In [12]:
labels = mnist_dataset[:,0]
labels

array([0., 5., 2., ..., 2., 3., 4.])

In [104]:
X = mnist_dataset[:,1:]
kmean = KMeans(n_clusters=10)
clustering = DBSCAN(eps=3, min_samples=2)
hierarchical = AgglomerativeClustering(n_clusters=20)

In [31]:
y_pred = kmean.fit_predict(X)
y_pred

array([9, 5, 8, ..., 2, 0, 3], dtype=int32)

In [35]:
# https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics.cluster
# https://scikit-learn.org/stable/modules/clustering.html#clustering-evaluation
# https://stackoverflow.com/questions/34047540/python-clustering-purity-metric

In [66]:
purity = getImpurity(y_pred, labels)
gini = getGini(y_pred, labels)

In [67]:
gini

0.884705300582197

In [63]:
purity

0.16

In [61]:
from sklearn.metrics.cluster import contingency_matrix
def getImpurity(membership, labels):
  CM = contingency_matrix(labels, membership)
  truthCount = np.sum(np.amax(CM,axis=1))
  
  # truthCount = np.sum(np.amax(CM), axis = 0 )
  totalCount = np.sum(CM)

  purity = truthCount / totalCount 

  return purity

In [64]:
def getGini(membership , labels) :
  CM = contingency_matrix(labels, membership)
  giniIndex = 0

  ginidensity = 0

  for _m in CM :
    total_dataPoint = np.sum(_m)
    # print("total_datapoint", total_dataPoint)
    ginidensity += total_dataPoint
    counter = 0
    for count in  _m :
      counter += pow(count/ total_dataPoint, 2)
    giniIndex += (1 - counter) * total_dataPoint
  gini_coeff = giniIndex/ ginidensity

  return gini_coeff

# PCA Dimenstionlity Reduction

In [78]:
from sklearn.decomposition import PCA , KernelPCA, SparsePCA
pca = PCA(n_components=100, random_state=200)


In [75]:
newX = pca.fit_transform(X, labels)
pca_kmeans = KMeans(20)

In [76]:
y_pca_predict = pca_kmeans.fit_predict(newX)

In [77]:
pca_gini = getGini(y_pca_predict, labels)
pca_purity =getImpurity(y_pca_predict, labels)
print(f"PCA Linear  Gini {pca_gini} Purity {pca_purity} ")

PCA Linear  Gini 0.9241143788424282 Purity 0.1174 


# PCA Dimenstionlity Reduction Gaussian PCA

In [148]:
pca_gaussian = KernelPCA(n_components=400, kernel="rbf", fit_inverse_transform=True, gamma=20)

In [149]:
newX_gausian = pca_gaussian.fit_transform(X, labels)
pca_gaussian_kmeans = KMeans(20)

In [150]:
y_pca_gaussain_predict = pca_gaussian_kmeans.fit_predict(newX_gausian)

In [151]:
pca_gauusian_gini = getGini(y_pca_gaussain_predict, labels)
pca_gaussian_purity =getImpurity(y_pca_gaussain_predict, labels)
print(f"PCA Linear  Gini {pca_gauusian_gini} Purity {pca_gaussian_purity} ")

PCA Linear  Gini 0.08419228842262003 Purity 0.9562 


In [152]:
pca_gaussian_purity2 =getImpurity( labels , y_pca_gaussain_predict)
print("puriy2", pca_gaussian_purity2)

puriy2 0.122


In [153]:
def harmonic(a,b):
  return 2*a*b / (a+b)

In [154]:
print(f"f1 {harmonic(pca_gaussian_purity, pca_gaussian_purity2)}")

f1 0.21639102207382674


# Sparse PCA

In [119]:
# pca_sparse= SparsePCA(n_components=30) 
# newX_sparse = pca_sparse.fit_transform(X, labels)
# pca_sparse_kmeans = KMeans(20)
# y_pca_sparse_predict = pca_sparse_kmeans.fit_predict(newX_sparse)
# pca_sparse_gini = getGini(y_pca_sparse_predict, labels)
# pca_sparse_purity =getImpurity(y_pca_sparse_predict, labels)
# print(f"PCA Linear  Gini {pca_sparse_gini} Purity {pca_sparse_purity} ")

In [157]:
aggla = FeatureAgglomeration(20)

In [164]:
agla_X = aggla.fit_transform(X)
agla_X.shape

(5000, 20)

In [160]:
kmean_aggla = KMeans(20)

In [161]:
aggla_pred = kmean_aggla.fit_predict(agla_X)

In [162]:
aggla_gini = getGini(aggla_pred, labels)
aggla_purity = getImpurity(aggla_pred, labels)
aggla_purity2 = getImpurity(labels, aggla_pred)

In [163]:
print(f"aggla_purity {aggla_purity} aggla_purity 2 {aggla_purity2}  f1 {harmonic(aggla_purity, aggla_purity2)}")

aggla_purity 0.205 aggla_purity 2 0.3912  f1 0.2690238175109024


# aggla feature + Kmeans kernel PCA


In [185]:
pca_gaussian = KernelPCA(n_components=20, kernel="linear", fit_inverse_transform=True, gamma=10)
newX_gausian_agla = pca_gaussian.fit_transform(agla_X, labels)
pca_gaussian_kmeans = KMeans(20)
y_pca_gaussain_predict = pca_gaussian_kmeans.fit_predict(newX_gausian_agla)
pca_gauusian_gini = getGini(y_pca_gaussain_predict, labels)
pca_gaussian_purity =getImpurity(y_pca_gaussain_predict, labels)
print(f"PCA Linear  Gini {pca_gauusian_gini} Purity {pca_gaussian_purity} ")
pca_gaussian_purity2 =getImpurity( labels , y_pca_gaussain_predict)
print("puriy2", pca_gaussian_purity2)
print(f"f1 {harmonic(pca_gaussian_purity, pca_gaussian_purity2)}")

PCA Linear  Gini 0.8786266507982413 Purity 0.1996 
puriy2 0.3694
f1 0.2591642882249561


Linear 50 gamma 5

In [186]:
pca_gaussian = KernelPCA(n_components=50, kernel="linear", fit_inverse_transform=True, gamma=5)
newX_gausian_agla = pca_gaussian.fit_transform(agla_X, labels)
pca_gaussian_kmeans = KMeans(20)
y_pca_gaussain_predict = pca_gaussian_kmeans.fit_predict(newX_gausian_agla)
pca_gauusian_gini = getGini(y_pca_gaussain_predict, labels)
pca_gaussian_purity =getImpurity(y_pca_gaussain_predict, labels)
print(f"PCA Linear  Gini {pca_gauusian_gini} Purity {pca_gaussian_purity} ")
pca_gaussian_purity2 =getImpurity( labels , y_pca_gaussain_predict)
print("puriy2", pca_gaussian_purity2)
print(f"f1 {harmonic(pca_gaussian_purity, pca_gaussian_purity2)}")

PCA Linear  Gini 0.8812417963389892 Purity 0.1998 
puriy2 0.3574
f1 0.25631198851399856


# RBF N_components = 50 Gamms 5 
## F1 0.34


In [187]:
pca_gaussian = KernelPCA(n_components=50, kernel="rbf", fit_inverse_transform=True, gamma=5)
newX_gausian_agla = pca_gaussian.fit_transform(agla_X, labels)
pca_gaussian_kmeans = KMeans(20)
y_pca_gaussain_predict = pca_gaussian_kmeans.fit_predict(newX_gausian_agla)
pca_gauusian_gini = getGini(y_pca_gaussain_predict, labels)
pca_gaussian_purity =getImpurity(y_pca_gaussain_predict, labels)
print(f"PCA Linear  Gini {pca_gauusian_gini} Purity {pca_gaussian_purity} ")
pca_gaussian_purity2 =getImpurity( labels , y_pca_gaussain_predict)
print("puriy2", pca_gaussian_purity2)
print(f"f1 {harmonic(pca_gaussian_purity, pca_gaussian_purity2)}")

PCA Linear  Gini 0.3107797981703676 Purity 0.806 
puriy2 0.2174
f1 0.3424358022278679


In [190]:
pca_gaussian = KernelPCA(n_components=50, kernel="linear", fit_inverse_transform=True, gamma=5)
newX_gausian_agla = pca_gaussian.fit_transform(X, labels)
pca_gaussian_kmeans = KMeans(20)
y_pca_gaussain_predict = pca_gaussian_kmeans.fit_predict(newX_gausian_agla)
pca_gauusian_gini = getGini(y_pca_gaussain_predict, labels)
pca_gaussian_purity =getImpurity(y_pca_gaussain_predict, labels)
print(f"PCA Linear  Gini {pca_gauusian_gini} Purity {pca_gaussian_purity} ")
pca_gaussian_purity2 =getImpurity( labels , y_pca_gaussain_predict)
print("puriy2", pca_gaussian_purity2)
print(f"f1 {harmonic(pca_gaussian_purity, pca_gaussian_purity2)}")

PCA Linear  Gini 0.9249904113025875 Purity 0.1294 
puriy2 0.2142
f1 0.1613357392316647


In [231]:
from sklearn.feature_selection  import mutual_info_classif
from sklearn.model_selection import  train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
 

In [194]:
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.10, random_state=42)

In [197]:
feature_scores = mutual_info_classif(X_test, y_test, random_state=0)

In [234]:
selector = SelectKBest(mutual_info_classif, k=30).fit(X_test, y_test)

In [235]:


train_X_transformed =selector.transform(X_train)



In [236]:
kmeans2 = KMeans(20)
pred  = kmeans2.fit_predict(X_train)

In [237]:
purity1 =getImpurity(pred, y_train)
print(f" Purity {purity1} ")
purity2 =getImpurity( y_train , pred)
print("puriy2", purity2)
print(f"f1 {harmonic(purity1, purity2)}")

 Purity 0.13 
puriy2 0.21866666666666668
f1 0.16305927342256216


# L1 regularization

In [239]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

In [252]:
sel_ = SelectFromModel(LogisticRegression(C=1, penalty='l1',solver='saga',max_iter=200),max_features=50)

In [253]:
sel_.fit(X_test, y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


SelectFromModel(estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                             fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=200, multi_class='auto',
                                             n_jobs=None, penalty='l1',
                                             random_state=None, solver='saga',
                                             tol=0.0001, verbose=0,
                                             warm_start=False),
                max_features=50, norm_order=1, prefit=False, threshold=None)

In [254]:
newX_train = sel_.transform(X_train)


In [255]:
kmeans2 = KMeans(20)
pred  = kmeans2.fit_predict(newX_train)

In [256]:
purity1 =getImpurity(pred, y_train)
print(f" Purity {purity1} ")
purity2 =getImpurity( y_train , pred)
print("puriy2", purity2)
print(f"f1 {harmonic(purity1, purity2)}")

 Purity 0.24822222222222223 
puriy2 0.45266666666666666
f1 0.32062692876770243
